# 04wk-2: 감성분석 파고들기 (2)

최규빈  
2024-09-19

<a href="https://colab.research.google.com/github/guebin/MP2024/blob/main/posts/02wk-1.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상

<https://youtu.be/playlist?list=PLQqh36zP38-xZ2p62reZCCgZsyMQLsOqY&si=bHkFTktvJIGoD0ni>

# 7. 데이터전처리2

In [401]:
def 데이터전처리하기2(examples):
    return 데이터전처리하기1(examples["text"], truncation=True)

In [402]:
데이터 = datasets.load_dataset('imdb')
전처리된데이터 = 데이터.map(데이터전처리하기2,batched=True)

Map: 100%|█████████████████████████████████████████████████████████████████████████| 25000/25000 [00:03<00:00, 8260.93 examples/s]

# 8. 데이터콜렉터

`-` 데이터콜렉터 사용방법

In [407]:
데이터콜렉터 = 동적패딩처리하기 = transformers.DataCollatorWithPadding(tokenizer=토크나이저,return_tensors='pt')
데이터콜렉터

In [408]:
작은덩어리_딕셔너리 = 전처리된데이터['train'][:4]
작은덩어리_딕셔너리.keys()

In [409]:
작은덩어리_리스트 = [{
    'label': 작은덩어리_딕셔너리['label'][i],
    'input_ids': 작은덩어리_딕셔너리['input_ids'][i],
    'attention_mask': 작은덩어리_딕셔너리['attention_mask'][i]
} for i in range(4)]

In [410]:
데이터콜렉터(작은덩어리_리스트)

In [411]:
인공지능(**데이터콜렉터(작은덩어리_리스트))

`-` 데이터콜렉터의 기능: batch_size 에 따라서 패딩시켜줌

# 9. 평가하기

`-` `accuracy.compute`의 기능

In [348]:
accuracy = evaluate.load("accuracy")

In [356]:
accuracy.compute(references=[0, 0, 0], predictions=[0, 1, 1])

`-` 함수내용

In [358]:
# def 평가하기(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     accuracy = evaluate.load("accuracy")
#     return accuracy.compute(predictions=predictions, references=labels)

In [417]:
def 평가하기(eval_pred):
    계산된숫자들_로짓, 실제정답 = eval_pred
    인공지능의예측 = np.argmax(계산된숫자들_로짓, axis=1)
    accuracy = evaluate.load("accuracy")
    return accuracy.compute(predictions=인공지능의예측, references=실제정답)

> `평가하기` 함수는 `eval_dataset`에 적용됨

# 10. 트레이너 세부지침

In [418]:
트레이너세부지침 = transformers.TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2, # 전체문제세트를 2번 공부하라..
    weight_decay=0.01,
    eval_strategy="epoch", 
    save_strategy="epoch", 
    load_best_model_at_end=True,
    push_to_hub=False,
)

> **옵션 설명:**
>
> 1.  **`output_dir="my_awesome_model"`**:
>     -   학습된 모델과 관련 파일(예: 체크포인트, 로그 등)이 저장될
>         디렉토리 경로를 지정합니다.
> 2.  **`learning_rate=2e-5`**:
>     -   학습률(learning rate)을 설정합니다. 모델이 가중치를 업데이트할
>         때 사용하는 스텝 크기로, 작은 값일수록 학습 속도가 느려지지만
>         안정성이 높습니다.
> 3.  **`per_device_train_batch_size=16`**:
>     -   **훈련** 시, 각 GPU 또는 CPU 디바이스에서 사용할 배치 크기를
>         설정합니다. 이 경우 각 디바이스에서 16개의 샘플을 한 번에
>         처리합니다.
> 4.  **`per_device_eval_batch_size=16`**:
>     -   **평가** 시, 각 GPU 또는 CPU 디바이스에서 사용할 배치 크기를
>         설정합니다. 평가 시에는 훈련과 같은 배치 크기를 사용하는 것이
>         일반적입니다.
> 5.  **`num_train_epochs=2`**:
>     -   전체 훈련 데이터셋을 몇 번 반복(에포크)할지 설정합니다.
>         여기서는 데이터셋 전체를 2번 학습하게 됩니다.
> 6.  **`weight_decay=0.01`**:
>     -   가중치 감쇠(weight decay)를 적용하여 모델의 가중치가 지나치게
>         커지지 않도록 제어합니다. 0.01의 값은 가중치가 조금씩 감소하게
>         하여 모델의 일반화 성능을 높이는 데 도움을 줄 수 있습니다.
> 7.  **`eval_strategy="epoch"`**:
>     -   평가 전략을 설정합니다. 여기서는 `epoch`으로 설정되어, 매
>         에포크가 끝날 때마다 평가가 진행됩니다.
>     -   다른 값으로는 `steps` (일정한 스텝마다 평가) 등이 있습니다.
> 8.  **`save_strategy="epoch"`**:
>     -   모델을 언제 저장할지 설정합니다. `epoch`으로 설정되면, 매
>         에포크가 끝날 때마다 체크포인트를 저장합니다.
>     -   다른 값으로는 `steps` (일정한 스텝마다 저장) 등이 있습니다.
> 9.  **`load_best_model_at_end=True`**:
>     -   학습이 끝난 후, 가장 성능이 좋았던 체크포인트를 불러옵니다.
>         평가 지표에 따라 가장 성능이 좋았던 모델을 자동으로 불러와
>         최종 모델로 사용하게 됩니다.
> 10. **`push_to_hub=False`**:
>     -   모델 학습이 끝난 후 Hugging Face Hub에 모델을 업로드할지
>         여부를 설정합니다. `False`로 설정하면 업로드하지 않습니다.
>     -   `True`로 설정하면 모델과 관련된 파일들이 Hugging Face Hub에
>         업로드되어 다른 사람들과 공유할 수 있습니다.

`-` lr = 학습률 = 답이 틀렸을 경우 혼내는 정도

-   정확한 느낌은 경사하강법을 이해해야함.

# 11. 트레이너

`-` 트레이너를 이용한 학습

In [421]:
트레이너 = transformers.Trainer(
    model=인공지능,
    args=트레이너세부지침,
    train_dataset=전처리된데이터['train'],
    eval_dataset=전처리된데이터['test'],
    tokenizer=토크나이저, # 왜 필요하지??
    data_collator=데이터콜렉터,
    compute_metrics=평가하기,
)
트레이너.train()

In [423]:
25000 / 16 

In [424]:
1563 * 2 

`-` 인공지능이 똑똑해졌을까?

In [425]:
인공지능.classifier.weight # 숫자들은 바뀌어 있음.. 

In [426]:
인공지능.pre_classifier.weight # 숫자들이 바뀌어 있음.. 

In [443]:
인공지능.to("cpu") # 인공지능이 지금 GPU에서 학습했거든요.. 다시 CPU로 내려주는 작업이 필요합니다

`-` 입력정보에 영화에 대한 부정적 평가를 넣는다면?

In [427]:
입력정보_원시텍스트 = "This movie was a huge disappointment."
정리된숫자_토큰화된자료 = 토크나이저(입력정보_원시텍스트,truncation=True,return_tensors='pt')
정리된숫자_토큰화된자료

In [431]:
인공지능(**정리된숫자_토큰화된자료)

In [432]:
계산된숫자_로짓 = 인공지능(**정리된숫자_토큰화된자료).logits.detach().numpy()
계산된숫자_로짓

In [433]:
출력정보_확률 = np.exp(계산된숫자_로짓) / np.exp(계산된숫자_로짓).sum()
출력정보_확률 # 0일확률, 1일확률

In [434]:
출력정보_확률.argmax() # 부정적 영화평가에 대한 인공지능의 예측

`-` 입력정보에 영화에 대한 긍정적 평가를 넣는다면?

In [436]:
입력정보_원시텍스트 = "This was a masterpiece."
정리된숫자_토큰화된자료 = 토크나이저(입력정보_원시텍스트,truncation=True,return_tensors='pt')
정리된숫자_토큰화된자료

In [437]:
인공지능(**정리된숫자_토큰화된자료)

In [438]:
계산된숫자_로짓 = 인공지능(**정리된숫자_토큰화된자료).logits.detach().numpy()
계산된숫자_로짓

In [439]:
출력정보_확률 = np.exp(계산된숫자_로짓) / np.exp(계산된숫자_로짓).sum()
출력정보_확률 # 0일확률, 1일확률

In [440]:
출력정보_확률.argmax()
계산된숫자_로짓.argmax() # 부정적 영화평가에 대한 인공지능의 예측

`-` 우리가 가져야할 생각: 신기하다 X // 노가다 많이 했구나.. O

# 12. 파이프라인

`-` 강인공지능?

> ref: <https://zdnet.co.kr/view/?no=20160622145838>

In [444]:
강인공지능 = transformers.pipeline("sentiment-analysis", model="my_awesome_model/checkpoint-1563")
print(강인공지능("This movie was a huge disappointment."))
print(강인공지능("This was a masterpiece."))

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.

[{'label': 'LABEL_0', 'score': 0.9885253310203552}]
[{'label': 'LABEL_1', 'score': 0.978060781955719}]